In [311]:
import json
import random

class CardType:
    def __init__(self, card_number, card, effect):
        self.card_number = card_number
        self.card = card
        self.effect = effect
        
    def play(self, target_player):
        pass

class Attack(CardType):
    def __init__(self, card_number, card, effect, damage):
        super().__init__(card_number, card, effect)
        self.damage = damage

    def play(self, target_player):
        target_player.take_damage(self.damage)

class Armor(CardType):
    def __init__(self, card_number, card, effect, protection):
        super().__init__(card_number, card, effect)
        self.protection = protection

    def play(self, target_player):
        target_player.get_protection(self.protection)

class Rejuvenate(CardType):
    def __init__(self, card_number, card, effect, healing):  
        super().__init__(card_number, card, effect)
        self.healing = healing 

    def play(self, player):
        player.get_healed(self.healing)


In [315]:
class Deck:
    def __init__(self):
        self.cards = []

    def generate_deck(self, size):
        card_numbers = list(range(1, size + 1))

        attack_cards = [
            "Kitana", "Rock", "Sledge Hammer", "Bat", "Crowbar", "Grenade",
            "Brick", "Broken Bottle", "Knife", "Pistol", "Torch", "Rocket Launcher",
            "Pepper Spray", "Shovel", "Axe", "Molotov", "Chainsaw", "Mace",
            "Blow Darts", "Magic Strike"
        ]
        armor_cards = [
            "Riot Shield", "Knights Garb", "Kevlar Vest", 
            "Inflatable Bumper Bubble", "Bomb Suit", "Magic Aura"
        ]
        rejuvenate_cards = [
            "Stim Pack", "Bandages", "Painkillers", "Magic Potion"
        ]

        magic_strike_count = 0
        magic_aura_count = 0
        magic_potion_count = 0
        for i in range(1, size + 1):
            card_type = random.choice([Attack, Armor, Rejuvenate])
            if card_type == Attack:
                card_name = random.choice(attack_cards)
                if card_name == "Magic Strike":
                    if magic_strike_count == 0:
                        magic_strike_count += 1
                        card = Attack(i, card_name, "Effect", 100)
                    else:
                        continue
                elif card_name in ["Pepper Spray", "Blow Darts"]:
                    card = Attack(i, card_name, "Effect", random.randint(1, 10))
                elif card_name in ["Rock", "Bat", "Brick", "Shovel"]:
                    card = Attack(i, card_name, "Effect", random.randint(10, 20))
                elif card_name in ["Crowbar", "Sledge Hammer"]:
                    card = Attack(i, card_name, "Effect", random.randint(20, 30))
                elif card_name in ["Kitana", "Switch Blade", "Broken Bottle", "Axe", "Mace"]:
                    card = Attack(i, card_name, "Effect", random.randint(30, 40))
                elif card_name in ["Chainsaw", "Torch"]:
                    card = Attack(i, card_name, "Effect", random.randint(40, 50))
                elif card_name == "Pistol":
                    card = Attack(i, card_name, "Effect", random.randint(50, 60))
                else:
                    card = Attack(i, card_name, "Effect", random.randint(60, 75))

            elif card_type == Armor:
                card_name = random.choice(armor_cards)
                if card_name == "Magic Aura":
                    if magic_aura_count == 0:
                        magic_aura_count += 1
                        card = Armor(i, card_name, "Effect", 100)
                    else:
                        continue
                elif card_name == "Inflatable Bumper Bubble":
                    card = Armor(i, card_name, "Effect", random.randint(1, 5))
                elif card_name == "Knights Garb":
                    card = Armor(i, card_name, "Effect", random.randint(5, 10))
                elif card_name == "Kevlar Vest":
                    card = Armor(i, card_name, "Effect", random.randint(10, 15))
                elif card_name == "Riot Shield":
                    card = Armor(i, card_name, "Effect", random.randint(15, 20))
                else:
                    card = Armor(i, card_name, "Effect", random.randint(20, 25))
            else:
                card_name = random.choice(rejuvenate_cards)
                if card_name == "Magic Potion":
                    if magic_potion_count == 0:
                        magic_potion_count += 1
                        card = Rejuvenate(i, card_name, "Effect", 100)
                    else:
                        continue
                elif card_name == "Bandages":
                    card = Rejuvenate(i, card_name, "Effect", random.randint(1, 10))
                elif card_name == "Painkillers":
                    card = Rejuvenate(i, card_name, "Effect", random.randint(10, 20))
                else:
                    card = Rejuvenate(i, card_name, "Effect", random.randint(20, 30))

            card.card_type = card_type.__name__
            self.cards.append(card.__dict__)
            
    def save_to_file(self, filename):
        with open(filename, 'w') as file:
            json.dump(self.cards, file, indent=4)

    def load_from_file(self, filename):
        with open(filename, 'r') as file:
            self.cards = json.load(file)

    def __str__(self):
        return json.dumps(self.cards, indent=4)
    
class Hand:
    def __init__(self, deck_file):
        self.cards = []
        self.deck_file = deck_file
        self.load_deck()

    def load_deck(self):
        with open(self.deck_file, 'r') as file:
            self.deck = json.load(file)

    def draw_card(self, player_name=None, enemy_name=None):
        drawn_card = None
        
        if not self.deck:
            print("Not enough cards in the deck!")
            return None

        while len(self.cards) < 5:
            drawn_card = self.deck.pop(0)
            value = drawn_card.get('damage', drawn_card.get('protection', drawn_card.get('healing', 0)))
            self.cards.append({
                'card': drawn_card['card'],
                'value': value
            })

            with open(self.deck_file, 'w') as file:
                json.dump(self.deck, file, indent=4)

            if len(self.cards) == 5:
                break
        
        magic_cards_encountered = set()
        magic_cards = ['Magic Strike', 'Magic Aura', 'Magic Potion']
        for card in self.cards:
            if card['card'] in magic_cards:
                if card['card'] not in magic_cards_encountered:
                    magic_cards_encountered.add(card['card'])
                    if player_name:
                        print(f"\n{player_name} has {card['card']}!")
                    elif enemy_name:
                        print(f"\n{enemy_name} has {card['card']}!")
                print(f"{player_name if player_name else enemy_name} has {card['card']}!")

        return drawn_card

class Player:
    def __init__(self, name, health, hand):
        self.name = name
        self.health = health
        self.hand = hand
        self.old_health = health
        self.protection = 0
        self.armor_active = False
        
    def play_card(self, card_name, target_player, player_name=None, enemy_name=None):
        selected_card = next((card for card in self.hand.cards if card['card'] == card_name), None)

        if selected_card:
            if selected_card['card'] in ['Kitana', 'Rock', 'Sledge Hammer', 'Bat', 'Crowbar', 'Dynamite',
                                            'Brick', 'Broken Bottle', 'Switch Blade', 'Pistol', 'Torch', 'Rocket Launcher',
                                            'Pepper Spray', 'Shovel', 'Axe', 'Molotov', 'Chainsaw', 'Mace',
                                            'Blow Darts', 'Magic Strike']:
                target_player.take_damage(selected_card['value'])
            elif selected_card['card'] in ['Riot Shield', 'Knights Garb', 'Kevlar Vest', 
                                            'Inflatable Bumper Bubble', 'Bomb suit', 'Magic Aura']:
                self.activate_armor(selected_card['value'])
            elif selected_card['card'] in ['Stim Pack', 'Bandages', 'Painkillers', 'Magic Potion']:
                self.get_healed(selected_card['value']) 

            return selected_card
    
    def activate_armor(self, protection):
        self.armor_active = True
        self.protection = max(protection, self.protection)
        print('Armor has been equipped!')
    
    def take_damage(self, damage):
        effective_damage = max(damage - self.protection, 0)  
        new_health = self.health - effective_damage
        if self.armor_active:
            print("Protection was used!")
            self.armor_active = False
        if new_health < self.old_health:  
            self.old_health = self.health  
            self.health = max(new_health, 0)
            if self.health <= 0: 
                print('RIP!\n')
            else:
                print('Attack activated!\n')
        
    def get_protection(self, protection):
        print('Armor has been equipped!\n')
        self.protection = max(protection, self.protection)
        
    def get_healed(self, healing):
        self.health += healing
        print('That feels much better!\n')


In [316]:
def initialize_player():
    player_deck = Deck()
    player_deck.generate_deck(50)
    player_deck.save_to_file('player_deck.json')
    player_name = input('Enter your player name: ')
    player_hand = Hand('player_deck.json')
    player_hand.draw_card(player_name)
    player = Player(player_name, 200, player_hand)
    
    return player

def initialize_enemy():
    enemy_deck = Deck()
    enemy_deck.generate_deck(50)
    enemy_deck.save_to_file('enemy_deck.json')
    enemy_name = 'The Boss'
    enemy_hand = Hand('enemy_deck.json')
    enemy_hand.draw_card(enemy_name)
    enemy = Player(enemy_name, 200, enemy_hand) 
    enemy_armored = False
    
    return enemy, enemy_armored
        
def round_of_play(player, enemy, player_name=None, enemy_name=None):
    enemy_armored = False
    
    while True:
        print("\nIt's your turn!")
        selected_card_name = input("Enter the name of the card you want to play: ")    
        selected_card = player.play_card(selected_card_name, enemy, player_name, enemy_name)

        if selected_card:
            print(f"{player_name if player_name else 'You'} play:", selected_card['card'])
            print("\nPlayer Health:", player.health)
            print(f"Enemy Health:", enemy.health)
            player.hand.cards.remove(selected_card) 
            print("\nDrawing next card...")
            print("A new card has been drawn!")
            break
        else:
            print("Card not found in your hand. Please enter a valid card name.")
    
    enemy_card = random.choice(enemy.hand.cards)
    enemy_card_name = enemy_card['card']
    
    print(f"\n{enemy_name} is up!")
    print(f"{enemy_name} plays:", enemy_card_name)
    enemy.play_card(enemy_card_name, player, enemy_name, player_name)
    if enemy.armor_active:
        enemy_armored = True
    else:
        enemy_armored = False
    print("\nPlayer Health:", player.health)
    print(f"Enemy Health:", enemy.health)
    enemy.hand.cards.remove(enemy_card)
    print("\nDrawing next card...")
    print("A new card has been drawn!")
    
    return enemy, enemy_armored


def display_info(player, enemy, enemy_armored):
    print('\nOpponent:', end=' ')
    print(enemy.name, end=' ')
    if enemy_armored:
        print("(Armored up)")
    else:
        print()

    print('Health:', enemy.health)
    print('Hand:')
    for card in enemy.hand.cards:
        print(f"{card['card']} - {card.get('value')} HP")
    
    print('\nPlayer:', end=' ')
    if player.armor_active:
        print(f"{player.name} (Armored up)")
    else:
        print(player.name)
        
    print('Health:', player.health)
    print('Hand:')
    for card in player.hand.cards:
        print(f"{card['card']} - {card.get('value')} HP")

def main():
    first_time = True
    player = initialize_player()
    enemy, enemy_armored = initialize_enemy() 
    
    while player.health > 0 and enemy.health > 0:
        if first_time:
            display_info(player, enemy, enemy_armored)
            first_time = False
        else:
            display_info(player, enemy, enemy_armored)
        
        enemy, enemy_armored = round_of_play(player, enemy, player.name, enemy.name) 

        if player.health <= 0:
            print("RIP! You lose.")
            break
        elif enemy.health <= 0:
            print("Congratulations! You win!")
            break
            
        player.hand.draw_card(player.name)
        enemy.hand.draw_card(enemy.name)

    play_again = input("\nDo you want to play again? (yes/no): ").lower()
    if play_again == 'no':
        print("Thank you for playing!")
    elif play_again != 'yes':
        print("Invalid input. Please enter 'yes' or 'no'.")
        return main()

if __name__ == "__main__":
    main()

Enter your player name: Noah

The Boss has Magic Aura!
The Boss has Magic Aura!

The Boss has Magic Potion!
The Boss has Magic Potion!

Opponent: The Boss 
Health: 200
Hand:
Grenade - 73 HP
Crowbar - 28 HP
Inflatable Bumper Bubble - 3 HP
Magic Aura - 100 HP
Magic Potion - 100 HP

Player: Noah
Health: 200
Hand:
Rock - 12 HP
Rock - 13 HP
Inflatable Bumper Bubble - 5 HP
Inflatable Bumper Bubble - 3 HP
Riot Shield - 17 HP

It's your turn!
Enter the name of the card you want to play: Rock
Attack activated!

Noah play: Rock

Player Health: 200
Enemy Health: 188

Drawing next card...
A new card has been drawn!

The Boss is up!
The Boss plays: Grenade

Player Health: 200
Enemy Health: 188

Drawing next card...
A new card has been drawn!

The Boss has Magic Aura!
The Boss has Magic Aura!

The Boss has Magic Potion!
The Boss has Magic Potion!

Opponent: The Boss 
Health: 188
Hand:
Crowbar - 28 HP
Inflatable Bumper Bubble - 3 HP
Magic Aura - 100 HP
Magic Potion - 100 HP
Grenade - 72 HP

Player: N

KeyboardInterrupt: Interrupted by user